In [28]:
import pandas as pd
import numpy as np
import seaborn as sns

# Get the data from the model

In [78]:
y_pred = pd.read_pickle('../data/pkl/y_pred_adv_test.pkl')
y_pred = y_pred[['GAME_ID', 'GAME_DATE', 'TEAM_NAME_h', 'TEAM_NAME_a', 'y_pred', 'PLUS_MINUS']]
y_pred.drop_duplicates(subset='GAME_ID', inplace=True)
y_pred['y_pred_spread'] = y_pred['y_pred'] * - 1
y_pred

,GAME_ID,GAME_DATE,TEAM_NAME_h,TEAM_NAME_a,y_pred,PLUS_MINUS,y_pred_spread
0,21900373,2019-12-13,Hawks,Pacers,0.046749,-10.0,-0.046749
6,21801139,2019-03-30,Pistons,Trail Blazers,3.024314,9.0,-3.024314
12,22000577,2021-03-11,Trail Blazers,Suns,0.848462,-6.0,-0.848462
18,22100417,2021-12-15,Cavaliers,Rockets,2.248967,35.0,-2.248967
24,22100862,2022-02-14,Bucks,Trail Blazers,8.223619,-15.0,-8.223619
...,...,...,...,...,...,...,...
5730,22100608,2022-01-10,Pistons,Jazz,1.464796,10.0,-1.464796
5736,22200676,2023-01-18,Nuggets,Timberwolves,9.150328,4.0,-9.150328
5742,21800242,2018-11-19,76ers,Suns,6.821165,5.0,-6.821165
5748,12100060,2021-10-15,Pistons,76ers,1.677697,4.0,-1.677697


# Get the historical betting data

In [79]:
bets = pd.read_pickle('../data/pkl/sbr_betting_data.pkl')
bets = bets.reset_index(drop=True)

## Fix the messed up column names from the pickle data

In [80]:
temp = ['Draft_Kings', 'Fanduel', 'Caesars', 'Pointsbet', 'Wynn', 'Betrivers']
merge_df = bets[['Game_Date', 'Team_Name', 'Home', 'Opponent', 'Points', 'Pct_of_Bets',
       'Opening_Spread', 'Opening_Odds', 'Betmgm', 'Betmgm_Odds']]
for i in range(12):
    if i % 2 == 0:
        name = temp[int(i/2)]
        temp_df = pd.DataFrame(bets.iloc[:, (i+10)]).rename(columns={f'{name}_Odds': f'{name}_Spread'}).reset_index(drop=True)
    else:
        name = temp[int(i/2)-1]
        temp_df = pd.DataFrame(bets.iloc[:, (i+10)]).rename(columns={f'{name}_Odds': f'{name}_Odds'}).reset_index(drop=True)
    merge_df = merge_df.merge(temp_df, left_index=True, right_index=True)
bets = merge_df.copy()
bets.rename(columns={'Betmgm': 'Betmgm_Spread'}, inplace=True)

In [81]:
bets.columns

Index(['Game_Date', 'Team_Name', 'Home', 'Opponent', 'Points', 'Pct_of_Bets',
       'Opening_Spread', 'Opening_Odds', 'Betmgm_Spread', 'Betmgm_Odds',
       'Draft_Kings_Spread', 'Draft_Kings_Odds', 'Fanduel_Spread',
       'Fanduel_Odds', 'Caesars_Spread', 'Caesars_Odds', 'Pointsbet_Spread',
       'Pointsbet_Odds', 'Wynn_Spread', 'Wynn_Odds', 'Betrivers_Spread',
       'Betrivers_Odds'],
      dtype='object')

In [82]:
sportsbooks_spreads = ['Opening_Spread','Betmgm_Spread', 'Draft_Kings_Spread', 'Fanduel_Spread', 
                       'Caesars_Spread', 'Pointsbet_Spread', 'Wynn_Spread', 'Betrivers_Spread']

sportsbooks_odds = ['Opening_Odds', 'Betmgm_Odds', 'Draft_Kings_Odds', 'Fanduel_Odds', 
                    'Caesars_Odds', 'Pointsbet_Odds', 'Wynn_Odds', 'Betrivers_Odds']

sportsbooks_spreads_odds = ['Betmgm_Spread', 'Betmgm_Odds',
       'Draft_Kings_Spread', 'Draft_Kings_Odds', 'Fanduel_Spread',
       'Fanduel_Odds', 'Caesars_Spread', 'Caesars_Odds', 'Pointsbet_Spread',
       'Pointsbet_Odds', 'Wynn_Spread', 'Wynn_Odds', 'Betrivers_Spread',
       'Betrivers_Odds']

In [83]:
bets = bets.replace(to_replace=['-', ''], value='1000')
bets = bets.replace(to_replace=['PK'], value='0')

In [84]:
bets[sportsbooks_spreads] = bets[sportsbooks_spreads].astype('float32')
bets[sportsbooks_odds] = bets[sportsbooks_odds].astype('int32')

In [85]:
bets = bets.replace(to_replace=[1000, '1000'], value=[np.nan, ''])
bets

,Game_Date,Team_Name,Home,Opponent,Points,Pct_of_Bets,Opening_Spread,Opening_Odds,Betmgm_Spread,Betmgm_Odds,...,Fanduel_Spread,Fanduel_Odds,Caesars_Spread,Caesars_Odds,Pointsbet_Spread,Pointsbet_Odds,Wynn_Spread,Wynn_Odds,Betrivers_Spread,Betrivers_Odds
0,2023-03-05,Phoenix,0,Dallas,130,56%,1.5,-110,-1.5,-115.0,...,-2.0,-108.0,-2.0,-110.0,-2.0,-110.0,-2.0,-110.0,-2.0,-112.0
1,2023-03-05,Dallas,1,Phoenix,126,44%,-1.5,-110,1.5,-105.0,...,2.0,-112.0,2.0,-110.0,2.0,-110.0,2.0,-110.0,2.0,-112.0
2,2023-03-05,Indiana,0,Chicago,125,53%,6.5,-110,5.5,-110.0,...,5.5,-108.0,5.0,-110.0,5.5,-110.0,5.5,-110.0,5.5,-110.0
3,2023-03-05,Chicago,1,Indiana,122,47%,-6.5,-110,-5.5,-110.0,...,-5.5,-112.0,-5.0,-110.0,-5.5,-110.0,-5.5,-110.0,-5.5,-112.0
4,2023-03-05,Golden State,0,L.A. Lakers,105,60%,-5.5,-110,-5.5,-105.0,...,-5.5,-118.0,-5.5,-110.0,-5.0,-110.0,-5.0,-110.0,-5.0,-109.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9861,2019-10-05,San Antonio,1,Orlando,89,,-5.5,-110,NaN,NaN,...,NaN,NaN,NaN,NaN,-6.5,-110.0,NaN,NaN,NaN,NaN
9862,2019-10-04,Houston,0,L.A. Clippers,109,,-4.5,-110,NaN,NaN,...,NaN,NaN,NaN,NaN,-5.0,-110.0,NaN,NaN,NaN,NaN
9863,2019-10-04,L.A. Clippers,1,Houston,96,,4.5,-110,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,-110.0,NaN,NaN,NaN,NaN
9864,2019-10-04,Indiana,0,Sacramento,132,,2.5,-110,NaN,NaN,...,NaN,NaN,NaN,NaN,5.5,-110.0,NaN,NaN,NaN,NaN


In [86]:
bets['Game_Date'] = pd.to_datetime(bets['Game_Date'])

In [87]:
home = bets[bets['Home'] == 1]
home = home[['Game_Date', 'Team_Name', 'Home', 'Opponent','Opening_Spread']]

In [88]:
y_pred['TEAM_NAME_a'].sort_values().unique()
team_name = ['Hawks', 'Celtics', 'Nets', 'Hornets', 'Bulls', 'Cavaliers', 'Mavericks', 'Nuggets', 'Pistons',
            'Warriors', 'Rockets', 'Pacers', 'Clippers', 'Lakers', 'Grizzlies', 'Heat', 'Bucks', 'Timberwolves',
            'Pelicans', 'Knicks', 'Thunder', 'Magic', '76ers', 'Suns', 'Trail Blazers', 'Kings', 'Spurs',
            'Raptors', 'Jazz', 'Wizards']
team_city = bets['Team_Name'].sort_values().unique().tolist()
map = {}
for i, city in enumerate(team_city):
    map[city] = team_name[i]
home = home.replace({'Team_Name': map, 'Opponent': map})

In [89]:
home

,Game_Date,Team_Name,Home,Opponent,Opening_Spread
1,2023-03-05,Mavericks,1,Suns,-1.5
3,2023-03-05,Bulls,1,Pacers,-6.5
5,2023-03-05,Lakers,1,Warriors,5.5
7,2023-03-05,Nets,1,Hornets,-7.5
9,2023-03-05,Magic,1,Trail Blazers,-1.5
...,...,...,...,...,...
9857,2019-10-05,Pacers,1,Kings,-3.0
9859,2019-10-05,Warriors,1,Lakers,2.0
9861,2019-10-05,Spurs,1,Magic,-5.5
9863,2019-10-04,Clippers,1,Rockets,4.5


# Compare the data

In [90]:
test = y_pred.merge(home, left_on=['GAME_DATE', 'TEAM_NAME_h', 'TEAM_NAME_a'], right_on=['Game_Date', 'Team_Name', 'Opponent'])

In [91]:
test = test.sort_values('GAME_DATE', ascending=False)

In [92]:
test[test['Opening_Spread'] > 0].head(25)

,GAME_ID,GAME_DATE,TEAM_NAME_h,TEAM_NAME_a,y_pred,PLUS_MINUS,y_pred_spread,Game_Date,Team_Name,Home,Opponent,Opening_Spread
70,22200942,2023-03-02,Wizards,Raptors,7.241134,11.0,-7.241134,2023-03-02,Wizards,1,Raptors,1.5
548,22200933,2023-03-01,Hornets,Suns,1.335198,-14.0,-1.335198,2023-03-01,Hornets,1,Suns,9.5
455,22200938,2023-03-01,Rockets,Grizzlies,-1.541310,-14.0,1.541310,2023-03-01,Rockets,1,Grizzlies,9.5
740,22200926,2023-02-28,Rockets,Nuggets,-9.407591,-21.0,9.407591,2023-02-28,Rockets,1,Nuggets,9.5
318,22200846,2023-02-11,Nets,76ers,-0.407621,-3.0,0.407621,2023-02-11,Nets,1,76ers,2.5
356,22200847,2023-02-11,Hornets,Nuggets,-2.967996,-14.0,2.967996,2023-02-11,Hornets,1,Nuggets,9.5
769,22200836,2023-02-10,Pacers,Suns,-6.378652,-13.0,6.378652,2023-02-10,Pacers,1,Suns,1.5
389,22200831,2023-02-09,Magic,Nuggets,-1.284936,11.0,1.284936,2023-02-09,Magic,1,Nuggets,6.5
60,22200827,2023-02-08,Rockets,Kings,3.548873,-2.0,-3.548873,2023-02-08,Rockets,1,Kings,7.5
235,22200817,2023-02-07,Nets,Suns,-0.971713,-4.0,0.971713,2023-02-07,Nets,1,Suns,4.5
